In [15]:
import mysql.connector
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta 

In [16]:
# Initialize the Faker library
fake = Faker()

# creating  mysql database connection

DB_CONFIG = {
    "host" :'localhost',
    "user" :'root',
    "password" :'dhanu@12345678',
    "database" : 'Crop_management'
}

#connecting to mysql database
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()

#creating table
# Create crops table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS crops (
    id INT AUTO_INCREMENT PRIMARY KEY,
    crop_name VARCHAR(255) NOT NULL,
    planting_date DATE,
    harvest_date DATE,
    growth_stage VARCHAR(255),
    pest_control_measures TEXT,
    yield_prediction INT
)
""")
conn.commit()

In [17]:
# List of sample crop names
crop_names = ["Wheat", "Rice", "Corn", "Soybean", "Barley", "Sugarcane", "Cotton", "Potato", "Tomato", "Lettuce"]

# List of possible growth stages
growth_stages = ["Seedling", "Vegetative", "Flowering", "Fruiting", "Maturity"]

# List of sample pest control measures
pest_control_measures_list = [
    "Use of organic pesticides",
    "Crop rotation",
    "Neem oil application",
    "Biological pest control",
    "Chemical pesticides",
    "Regular field monitoring",
]

In [18]:
# Function to generate random data for crops
def generate_data():
    crop_name = random.choice(crop_names)
    planting_date = fake.date_between(start_date="-2y", end_date="today")  # Planting in last 2 years
    harvest_date = planting_date + timedelta(days=random.randint(60, 180))  # Harvest after 2-6 months
    growth_stage = random.choice(growth_stages)
    pest_control = random.choice(pest_control_measures_list)
    yield_prediction = random.randint(500, 5000)  # Yield in kg
    
    return (crop_name, planting_date, harvest_date, growth_stage, pest_control, yield_prediction)

In [19]:
# Batch insert records in chunks
batch_size = 100000 # Insert 10,000 at a time for efficiency
total_records = 20000000

for i in range(0, total_records, batch_size):
    data_batch = [generate_data() for _ in range(batch_size)]
    
    try:
        # Execute batch insert
        cursor.executemany("""
            INSERT INTO crops (crop_name, planting_date, harvest_date, growth_stage, pest_control_measures, yield_prediction)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, data_batch)
        conn.commit()
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        print("Reconnecting to the database...")
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()
        cursor.executemany("""
            INSERT INTO crops (crop_name, planting_date, harvest_date, growth_stage, pest_control_measures, yield_prediction)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, data_batch)
        conn.commit()
    
    print(f"{i + batch_size} records inserted...")

print("✅ Data insertion completed successfully!")

# Close the database connection
cursor.close()
conn.close()

100000 records inserted...
200000 records inserted...
300000 records inserted...


KeyboardInterrupt: 

In [ ]:
import pandas as pd

# Re-establish the MySQL connection
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()

# Define your SQL query
query = "SELECT * FROM crops"

# Use pandas to read the SQL query result into a DataFrame
df = pd.read_sql(query, conn)

# Export the DataFrame to a CSV file
df.to_csv("output3.csv", index=False)

# Close the database connection
cursor.close()
conn.close()

C:\Users\dhans\AppData\Local\Temp\ipykernel_8612\2842257181.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
